In [25]:
!wget -c 'https://raw.githubusercontent.com/omyfish/deep-learning/master/Char%20RNN/data/anna.txt'

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [5]:
with open('anna.txt', 'r') as f:
    text = f.read()
    
print(text[:100])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin


## Tokenization

Create a couple dictionaries to convert char to and from integers. Encoding characters as integers make it easier to used as input in the network

In [139]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers

char = tuple(set(text)) ## this will read out the distinct character
int2char = dict(enumerate(char)) ## maps integers to characters
char2int = {ch: ii for ii, ch in int2char.items()}

#encode the text
encoded = np.array([char2int[ch] for ch in text])

In [140]:
encoded[:100]

array([79, 49, 16, 75, 62, 42, 68, 17, 57, 81, 81, 81, 47, 16, 75, 75,  1,
       17, 11, 16, 58, 56, 14, 56, 42, 63, 17, 16, 68, 42, 17, 16, 14, 14,
       17, 16, 14, 56, 36, 42, 82, 17, 42, 69, 42, 68,  1, 17, 52, 39, 49,
       16, 75, 75,  1, 17, 11, 16, 58, 56, 14,  1, 17, 56, 63, 17, 52, 39,
       49, 16, 75, 75,  1, 17, 56, 39, 17, 56, 62, 63, 17, 21, 50, 39, 81,
       50, 16,  1, 78, 81, 81, 43, 69, 42, 68,  1, 62, 49, 56, 39])

## Pre-processing the data
Our LSTM expect an input that is One-hot-encoding


In [277]:
def one_hot_encode(arr, n_labels):
    #initiate the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    #fill the appropriate element with 1    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    #finally, reshape it back to the orignal array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [278]:
arr = np.array([3,2,1])
one_hot = one_hot_encode(arr, 8)
print(one_hot)

[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]


In [279]:
def get_batches(arr, batch_size, seq_length):
    num_batch_total = batch_size * seq_length
    n_batches = len(arr) // num_batch_total
    arr = arr[:(n_batches * num_batch_total)]
    arr = arr.reshape((batch_size, -1))

    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:,:-1],y[:,-1] = x[:,1:], arr[:, n+seq_length]
        except IndexError:
            y[:,:-1],y[:,-1] = x[:,1:], arr[:, 0]
        yield x, y

In [280]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)


In [281]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[79 49 16 75 62 42 68 17 57 81]
 [63 21 39 17 62 49 16 62 17 16]
 [42 39 22 17 21 68 17 16 17 11]
 [63 17 62 49 42 17 59 49 56 42]
 [17 63 16 50 17 49 42 68 17 62]
 [59 52 63 63 56 21 39 17 16 39]
 [17 60 39 39 16 17 49 16 22 17]
 [38  3 14 21 39 63 36  1 78 17]]

y
 [[49 16 75 62 42 68 17 57 81 81]
 [21 39 17 62 49 16 62 17 16 62]
 [39 22 17 21 68 17 16 17 11 21]
 [17 62 49 42 17 59 49 56 42 11]
 [63 16 50 17 49 42 68 17 62 42]
 [52 63 63 56 21 39 17 16 39 22]
 [60 39 39 16 17 49 16 22 17 63]
 [ 3 14 21 39 63 36  1 78 17 67]]


In [282]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [322]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=125, n_layers=2, drop_prop=0.5, lr = 0.01):
        super().__init__()
        # hyp
        self.drop_prop = drop_prop
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lr = lr
        
        #creating char dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(tokens))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # define layer of the mdel
        self.lstm = nn.LSTM(input_size = len(tokens), hidden_size = n_hidden,num_layers = n_layers, dropout = drop_prop, batch_first=True)
        self.dropout = nn.Dropout(drop_prop)
        self.fc = nn.Linear(n_hidden, len(tokens))
    def forward(self, x, hidden):
        r_out, hidden = self.lstm(x, hidden)
        r_out = self.dropout(r_out)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = r_out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        return hidden

In [323]:
tokens = tuple(set(text))
n_hidden = 512
n_layers = 2
dropoutrate = 0.5
net = CharRNN(tokens)

print(net)

CharRNN(
  (lstm): LSTM(83, 125, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=125, out_features=83, bias=True)
)


## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [328]:
import torch.optim as optim

def train(net, data, epochs=10, batch_size=10,seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 

    Arguments
    ---------

    net: CharRNN network
    data: text data to train the network
    epochs: Number of epochs to train
    batch_size: Number of mini-sequences per mini-batch, aka batch size
    seq_length: Number of character steps per mini-batch
    lr: learning rate
    clip: gradient clipping
    val_frac: Fraction of data to hold out for validation
    print_every: Number of steps for printing training and validation loss

    '''    
    net.train()
    optimizer = optim.Adam(net.parameters(), lr = lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation dat
    val_idx = int(len(data) * (1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if (train_on_gpu):
        net.cuda()
    
    count = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        for x, y in get_batches(data, batch_size, seq_length):
            count += 1
            #One-hot encode our data and make them torch tensor
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            targets = targets.view(batch_size*seq_length).long()
            loss = criterion(output, targets)
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()
            
            if count % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size * seq_length).long())
                    val_losses.append(val_loss.item())
                net.train() # reset to train mode after iterating the validation 
                
                print('Epochs {}/{}'.format(e+1, epochs),
                     'Step: {}'.format(count),
                     'Loss: {:.4f}...'.format(loss.item()),
                     'Validation Loss {:.4f}...'.format(np.mean(val_losses)))
                
            

In [329]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epochs 1/20 Step: 10 Loss: 3.9972... Validation Loss 3.8030...
Epochs 1/20 Step: 20 Loss: 3.2697... Validation Loss 3.1886...
Epochs 1/20 Step: 30 Loss: 3.2344... Validation Loss 3.1570...
Epochs 1/20 Step: 40 Loss: 3.1879... Validation Loss 3.1466...
Epochs 1/20 Step: 50 Loss: 3.2035... Validation Loss 3.1420...
Epochs 1/20 Step: 60 Loss: 3.1801... Validation Loss 3.1396...
Epochs 1/20 Step: 70 Loss: 3.1587... Validation Loss 3.1377...
Epochs 1/20 Step: 80 Loss: 3.1661... Validation Loss 3.1357...
Epochs 1/20 Step: 90 Loss: 3.1726... Validation Loss 3.1342...
Epochs 1/20 Step: 100 Loss: 3.1571... Validation Loss 3.1326...
Epochs 1/20 Step: 110 Loss: 3.1551... Validation Loss 3.1312...
Epochs 1/20 Step: 120 Loss: 3.1350... Validation Loss 3.1297...
Epochs 1/20 Step: 130 Loss: 3.1532... Validation Loss 3.1276...
Epochs 2/20 Step: 140 Loss: 3.1307... Validation Loss 3.1256...
Epochs 2/20 Step: 150 Loss: 3.1448... Validation Loss 3.1233...
Epochs 2/20 Step: 160 Loss: 3.1345... Validation 

Epochs 10/20 Step: 1290 Loss: 2.2344... Validation Loss 2.1594...
Epochs 10/20 Step: 1300 Loss: 2.2314... Validation Loss 2.1566...
Epochs 10/20 Step: 1310 Loss: 2.2363... Validation Loss 2.1548...


KeyboardInterrupt: 

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [330]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [422]:
import torch.nn.functional as F

def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        if h == None:
            h = net.init_hidden(1)
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if (train_on_gpu):
            inputs = inputs.cuda()
            
        # detach hidden state from history
        h = tuple([each.data for each in h ])
        # get the output of the model
        out, h = net(inputs, h)
        
        # get the character probabilities
        p = F.softmax(out, dim=1).data
        
        if (train_on_gpu):
            p = p.cuda()
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.to('cpu').numpy().squeeze()
    

        # select the likely next character with some element of randomness
        p = p.to('cpu').numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
       
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [423]:
a = predict(net, 'T')


### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [429]:
def sample(net, size, prime='The', top_k=None):
    if (train_on_gpu):
        net.cuda()
    else:
        net.cpu()
        
    net.eval() #evalate net
    
    #First off, run throught the prime char
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k = top_k)
        chars.append(char)
        
    #Now pass in the previous char and get new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    
    return ''.join(chars)
        
    

In [430]:
sample(net, 1000, prime ='Anna', top_k=5)

'Annahhthhhoohhthhioitoohoehhhohieihoheithhohoehhhhehhtothhthhhhoeieiihhehtohihhehiioehthehtethhhhehhhhhoioetoihhiehohitthhehheeeihheiteiehoiothiehhhhhehiioiheheohoieehhhiiehhtohehhhoiteihhohtehohhhhetehhotheeihhothoohothhhhoohteiihethohhhhhthhhooehhhhohhhioohehehthihhhhhttthihthhthihhioeeheheihhoihohhhhheohhthhitohhhehhihoehiihthohhhhhihhihhihohhihheithehohhohoothhhehehtehihhthhethihheetheohhhheehthhheehotethhotohhthiiihihoohhohoeeiiohththhthiehhhhhhtheotthitihteoeteiehhhhehhehoihttheoitehhotheehhohheethhhihhihhoeeohthhehtihoththhetthiihhthehhhhhthhioehhtiohttohhoehihetthietthhihhhtohohetihhhhoihhthttihoehehhtohoootothhhiihehhtehheohhhoiiehhhhhihthothhhtehhhhiooethththhhihhtiehhehhtohhhiiohohhihhhotiohehhehhhhheeetehehehtiiehhhhhteehtitthhehhihethheehhehhteeoohhhteoethhehhettehhetetoohihheihthhihthohhhieihhihehetohthehihhhoohhotthhoihheoehoihhhhhhohohhhthhhiihhtieehteethioihhthiihtoohhietohhhohetihhehhtheotteithhhhehhiiooieihehhohhihhiheehthithhhehehhohhhtheihhihiiiethehieohttohhhtoio

## Loading a checkpoint

In [433]:
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], checkpoint['n_hidden'], checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [434]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin saidiehehihohehehhhhhhhhhthhehhhhooethoohhoiheheihthtthhhheitoehhhtheihohtehheeihehehiohhteheieehhtheetteehhthehehoeoeitehhhhiitootthhhhhhtthhtiohhhhhhhetothhoihiteiihtihhhihhthhheeehheioehihioteiehhohhthiihhehiihhehoehhhhhhtteeohhhhithhhhhehoihhhhhhihhhtieiitthhteihhteththohhihehheethhhhhitihtihhheehhehoheeoohhhieiohhothhhhhhohoeihtihhiothoheheiiihtheehihhoeehhehhhihththhhhhihhhihheoheohhhoiohhihhtihihtehhhohiehehheeehhtthhhohhhthhthhhthhhhihoeohhtoothihttihhhihitietiteeteetheitithtieotihhehohiioiohtteihhhehiooheehooeehhhtiihihtohhhththohoooheeoohotehteohithhhieheeotehthteeohohhhhhteihiihtihohehihiohhhththehhhothiiiehhhhhohhheeohoooootihohhhthhthoeheohhhththohethheoeehhhhhihothohothihhiiheehhhhhhohhehheehhiiehhihohohththothetthhtthhhietthhotoehhetoihotohhehtthheehehhetehheehthihhhhehhoihoehhihihhhhihihtthetohhihitoothtihtoothhihhhhhiettioehhohohehhohhohheotteohtiehtihhhhotttiheheohohhthooihoohheetheeieeiieehhhihhehhheeethoieththhthhieohhehehoteohtohihohehhihehhhtthothitooehh